# Get main content of news

To get main content of news, we will use a library called newspaper:

Firstly, install the library by running the below code:



In [1]:
!curl https://raw.githubusercontent.com/codelucas/newspaper/master/download_corpora.py | python3
!pip install newspaper3k

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   657  100   657    0     0   1483      0 --:--:-- --:--:-- --:--:--  1486
[nltk_data] Downloading package brown to
[nltk_data]     /home/nguyenhieuec/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/nguyenhieuec/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /home/nguyenhieuec/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/nguyenhieuec/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/nguyenhieuec/nltk_data...
[nltk_data]   Package wordnet is al

Import the library



In [2]:
from newspaper import fulltext
import requests

Try to get content of the below link by



In [ ]:
url = "https://seekingalpha.com/news/3454711-facebook-broke-privacy-laws-exposing-user-data-canada-says"
html = requests.get(url).text
text = fulltext(html)
text

It's really easy, isn't it? However, when you crawl thousands of articles, the

# Adding proxies

The purpose of adding proxies to the request is to get the content indirectly via other IPs. You will change the proxy in each time you call request, the seekingalpha will be not able to detect and ban your IP anymore.

## Getting proxies

How can we get the list of free proxies?

To get the list of proxies, we need to get the html of the page and parse it with BeautifulSoup library. Firstly, let's install and import essential libraries



In [3]:
# !pip install lxml
import random

import requests
from bs4 import BeautifulSoup

The following function retrieves a list of proxies, please finish it by following instructions:



In [4]:
def get_proxies():
### START CODE HERE
# Create a GET request with url is https://free-proxy-list.net/
# and header User-Agent: Mozilla/5.0
    res = requests.get('https://free-proxy-list.net/', headers={'User-Agent':'Mozilla/5.0'})

    # Create a BeautifilSoup object with html from 'res'
    # and using 'lxml' as HTML parser
    soup = BeautifulSoup(res.text,"lxml")

    proxies = []

    # Get all rows by select "tbody tr"
    rows = soup.select("tbody tr")
    for row in rows:
    # Select all row values by "td"
        values = row.select("td")
        # Build the proxy with the syntax <values[0].text>:<values[1].text>
        # then append to 'proxies'
        proxy = values[0].text + ":" + values[1].text
        proxies.append(proxy)

    ### END CODE HERE
    return proxies

Try getting the list of proxies with the above function



In [ ]:
get_proxies()

## Adding proxies to request

After getting the list of proxies, we will pick a random proxy and add it to the request we call:



In [ ]:
# Url to get content
url = "https://seekingalpha.com/news/3454711-facebook-broke-privacy-laws-exposing-user-data-canada-says"
# Get list of proxies
proxies = get_proxies()
# Pick a random proxy
proxy = proxies[random.randint(0, len(proxies)-1)]
# Build proxy dictionary
proxy_dict = {
"http": "http://"+proxy,
"https": "https://"+proxy
}
# Request with proxy
html = requests.get(url, headers={'User-Agent':'Mozilla/5.0'}, proxies=proxy_dict).text
text = fulltext(html)
text

The proxies we got are all free, so it will not be stable (it usually throws error), we need to check the status of request to know whether it's success or not. When it get the status 200 like the below example, the request is success, otherwise you need to retry with another proxy:



In [ ]:
requests.get(url, headers={'User-Agent':'Mozilla/5.0'}).status_code

Besides, we should not use the same proxy list for all of your requests, we will check and request new proxies every 2 minutes:



In [5]:
from datetime import datetime
last_request_time = datetime.now()
current_proxies = []
def get_current_proxies():
    global last_request_time, current_proxies
    # If last request is in the last 2 minutes, do not request proxy list
    if (datetime.now() - last_request_time).seconds < 120 and len(current_proxies) > 0:
        return current_proxies
    # Get new proxy list
    current_proxies = get_proxies()
    last_request_time = datetime.now()
    return current_proxies

The next function receives a url and return the content of this url. Following the below instruction, complete the function



In [11]:
from urllib3.exceptions import ProxyError as ProxyError
from urllib3.exceptions import NewConnectionError
# from urllib3.exceptions import MissingSchema as MissingSchemaError
from urllib3.exceptions import HTTPError as BaseHTTPError
import requests.exceptions  as rex
def get_content_from_url(url):
    global no_crawled_content
    global no_err
    global url_err
    url_err = []
    ### START CODE HERE
    # Repeat until getting content
    while True:
    # Get list of proxies
        proxies = get_current_proxies()
        # Pick a random proxy and build proxy dictionary
        proxy = proxies[random.randint(0, len(proxies)-1)]
        proxy_dict = {
             "http": "http://"+proxy,
        }
        # Request 'url' with proxy (Should catch error when you request)
        try:
            response = requests.get(url, headers={'User-Agent':'Mozilla/5.0'}, proxies=proxy_dict)
        except (ProxyError, NewConnectionError, ConnectionError, requests.ConnectionError, rex.MissingSchema) as e:
#             no_err += 1
#             print(no_err, ". Error", url)
#             url_err.append(url)
            print("Cannot connect to proxy", proxy)
            print(e.args)
            continue
        # Check whether response status code is 200
        if response.status_code == 200:
        # Parse content from response html
            no_crawled_content += 1
            print(no_crawled_content, ". Crawled", url)
            return url, fulltext(response.text)
### END CODE HERE

In [13]:
requests.exceptions?

Test the function with the code below



In [ ]:
no_crawled_content = 0
get_content_from_url("https://seekingalpha.com/news/3454711-facebook-broke-privacy-laws-exposing-user-data-canada-says")

# Crawling content from urls with multi-thread

We will use the



In [ ]:
'''
Dataset [Seeking Alpha news urls] is mounted
and stored in [input/seeking-alpha-news-urls-5cc65ca5cd14d00011a84326] of current machine.

Please check your just-mounted dataset by running
bellow command:
# '''

# import os
# data_path_326 = "input/seeking-alpha-news-urls-5cc65ca5cd14d00011a84326"
# # Use "data_path_326" as the list of items on this dataset

# os.listdir(data_path_326)

Get the csv path:



In [ ]:
# csv_path = os.path.join(data_path_326, 'seeking_alpha.csv')

Read the list of urls



In [ ]:
!ls

In [7]:
import pandas as pd
import csv

In [8]:
col_names = ["url", "title", "stock", "timestamp", "col5", "col6"]

url_df = pd.read_csv('googl.csv',escapechar='\"', delimiter=',', quoting=csv.QUOTE_NONE, error_bad_lines=False,encoding = 'unicode_escape', names=col_names)
url_df.head()

,url,title,stock,timestamp,col5,col6
0,url,title,stock,timestamp,NaN,NaN
1,https://seekingalpha.com/news/3467965-google-m...,"Google Maps adds popular menu items, GOOGL",1559233807,NaN,NaN,NaN
2,https://seekingalpha.com/news/3467751-google-b...,Google bans apps that facilitate marijuana sal...,1559208044,NaN,NaN,NaN
3,https://seekingalpha.com/news/3467360-google-t...,Google temps outnumber full-time workers - NYT...,1559119152,NaN,NaN,NaN
4,https://seekingalpha.com/news/3466946-google-i...,Google to invest %u20AC600M in Finnish data ce...,1558949797,NaN,NaN,NaN


We will add one more column to the dataframe called



In [9]:
from multiprocessing.dummy import Pool as ThreadPool

In [ ]:

pool = ThreadPool(8)
results = pool.map(get_content_from_url, ["https://seekingalpha.com/news/3454711-facebook-broke-privacy-laws-exposing-user-data-canada-says"])
pool.close()
pool.join()

In [10]:
no_crawled_content = 0
pool = ThreadPool(32)
results = pool.map(get_content_from_url, list(url_df.url))
pool.close() 
pool.join()

1 . Crawled https://seekingalpha.com/news/3390950-big-google-push-auto-industry
2 . Crawled https://seekingalpha.com/news/3412271-google-makes-another-silicon-valley-land-grab
3 . Crawled https://seekingalpha.com/news/3424436-digital-giants-crosshairs-eu-tax
4 . Crawled https://seekingalpha.com/news/3445577-final-vote-eus-controversial-copyright-law
5 . Crawled https://seekingalpha.com/news/3432866-apple-increased-autonomous-testing-2018-update
6 . Crawled https://seekingalpha.com/news/3380302-intercept-google-using-baidu-search-results-re-enter-china
7 . Crawled https://seekingalpha.com/news/3406292-alphabet-kicks-two-day-health-conference
8 . Crawled https://seekingalpha.com/news/3451093-alphabet-starts-2019-strong-u-s-spending-otr
9 . Crawled https://seekingalpha.com/news/3395557-sequoia-fund-added-facebook-stake-q3
10 . Crawled https://seekingalpha.com/news/3413959-stifel-google-bid-red-hat
11 . Crawled https://seekingalpha.com/news/3421537-google-roundup-assistants-busy-ces19
12 .

89 . Crawled https://seekingalpha.com/news/3386961-blackberry-patent-android-case-ruled-invalid
90 . Crawled https://seekingalpha.com/news/3417965-eu-court-kicks-yelp-google-appeal
91 . Crawled https://seekingalpha.com/news/3435135-amazons-ad-business-will-take-google-report
92 . Crawled https://seekingalpha.com/news/3415724-google-speeds-plus-shutdown-new-bug-exposes-data
93 . Crawled https://seekingalpha.com/news/3395536-youtube-tv-boosts-cloud-dvr-features-user-feedback
94 . Crawled https://seekingalpha.com/news/3383075-amazons-smart-speaker-share-fell-q2-google-grew
95 . Crawled https://seekingalpha.com/news/3442397-microsoft-brings-pc-games-xbox
96 . Crawled https://seekingalpha.com/news/3442348-t-confirms-directv-now-changes-viacom-discovery-amc-tumble
97 . Crawled https://seekingalpha.com/news/3411628-former-google-cloud-head-internal-clashes
98 . Crawled https://seekingalpha.com/news/3385469-buzzfeed-major-tech-firms-gathering-discuss-election-defense-strategy
99 . Crawled http

176 . Crawled https://seekingalpha.com/news/3393060-big-spotlight-stay-big-tech
177 . Crawled https://seekingalpha.com/news/3404872-traditional-ad-industry-facing-shortage-internet-threats
178 . Crawled https://seekingalpha.com/news/3392877-reuters-google-acknowledge-privacy-mistakes-senate
179180  . Crawled https://seekingalpha.com/news/3446614-report-hud-also-probing-google-twitter-ad-discrimination
. Crawled https://seekingalpha.com/news/3395875-google-drops-race-10b-pentagon-cloud-contract
181 . Crawled https://seekingalpha.com/news/3465407-google-launches-999-ar-headset
182 . Crawled https://seekingalpha.com/news/3401921-tech-giants-may-face-billions-new-taxes
183 . Crawled https://seekingalpha.com/news/3434484-google-announces-gaming-keynote-next-month
184 . Crawled https://seekingalpha.com/news/3465400-former-apple-consumer-chip-engineers-exit-google-report
185 . Crawled https://seekingalpha.com/news/3414759-france-will-tax-digital-giants-2019
186 . Crawled https://seekingalpha.

262 . Crawled https://seekingalpha.com/news/3422476-google-eyes-600m-minnesota-data-center-powered-xcel-energy-wind-farms
263 . Crawled https://seekingalpha.com/news/3422289-alphabet-board-sued-misconduct-coverup
264 . Crawled https://seekingalpha.com/news/3419838-will-faangs-perform-2019
265 . Crawled https://seekingalpha.com/news/3433972-alphabet-wants-cut-toronto-smart-city
266 . Crawled https://seekingalpha.com/news/3384252-google-imax-scrap-vr-camera-project
267 . Crawled https://seekingalpha.com/news/3410742-waymo-big-headstart-autonomous-cars
268 . Crawled https://seekingalpha.com/news/3383867-google-launch-answer-echo-show-holidays
269 . Crawled https://seekingalpha.com/news/3383796-google-admits-unauthorized-location-tracking
270 . Crawled https://seekingalpha.com/news/3433588-google-fights-satellite-operators-5g-spectrum-information
271 . Crawled https://seekingalpha.com/news/3451598-cnbc-facebook-building-entry-voice-assistants
272 . Crawled https://seekingalpha.com/news/345

349 . Crawled https://seekingalpha.com/news/3418905-rbc-google-hardware-will-hit-20b-three-years
350 . Crawled https://seekingalpha.com/news/3445777-ftc-orders-broadband-providers-produce-privacy-practices
351 . Crawled https://seekingalpha.com/news/3445764-google-launching-local-experiments-project-axios
352 . Crawled https://seekingalpha.com/news/3367389-ft-alphabet-joins-300m-round-electric-scooter-startup
353 . Crawled https://seekingalpha.com/news/3396789-u-s-senators-want-answers-investigation-google-issue
354 . Crawled https://seekingalpha.com/news/3422220-amazon-comes-facebooks-ad-money-cowen
355 . Crawled https://seekingalpha.com/news/3406967-cnbc-comcast-plans-streaming-box-net-customers
356 . Crawled https://seekingalpha.com/news/3357479-morgan-stanley-selects-30-stocks-three-year-sideways-market
357 . Crawled https://seekingalpha.com/news/3369272-nomura-says-buy-alphabet-facebook-regulatory-scrutiny
358 . Crawled https://seekingalpha.com/news/3362386-wsj-congress-like-googl

...

1411 . Crawled https://seekingalpha.com/news/3205628-google-hires-airbnb-executive-self-driving-car-project
1412 . Crawled https://seekingalpha.com/news/1910715-google-ups-search-rankings-of-encrypted-websites-acquires-directr
1413 . Crawled https://seekingalpha.com/news/2947326-rough-time-hedge-fund-hotels
1414 . Crawled https://seekingalpha.com/news/3198006-youtube-releases-live-stream-viewer-numbers-party-conventions
1415 . Crawled https://seekingalpha.com/news/2276326-bloomberg-google-twitter-strike-deal-to-add-tweets-to-results
1416 . Crawled https://seekingalpha.com/news/2032855-google-launches-android-5_0-new-nexus-mobile-and-tv-hardware
1417 . Crawled https://seekingalpha.com/news/3203430-google-abandons-hangouts-air-google-pushes-users-youtube-live
1418 . Crawled https://seekingalpha.com/news/2636235-upgrade-target-hikes-follow-googles-earnings-shares-up-over-13-percent
1419 . Crawled https://seekingalpha.com/news/2578485-google-launching-youtube-gaming-positive-reviews-for-no

1487 . Crawled https://seekingalpha.com/news/1660193-googles-nest-labs-hit-by-smoke-alarm-hitch
1488 . Crawled https://seekingalpha.com/news/2028575-schmidt-defends-google-against-eu-calls-amazon-biggest-search-rival
1489 . Crawled https://seekingalpha.com/news/2745516-stick-market-darlings-rbc-analysts-say
1490 . Crawled https://seekingalpha.com/news/2449876-google-rises-in-spite-of-q1-miss-strong-dollar-a-major-headwind
1491 . Crawled https://seekingalpha.com/news/1817603-more-from-google-i-o-android-tv-chrome-os-gmail-drive
1492 . Crawled https://seekingalpha.com/news/2179735-wsj-google-testing-buy-button-mulling-amazon-prime-like-program
1493 . Crawled https://seekingalpha.com/news/2740596-google-set-life-sciences-team-free-within-alphabet
1494 . Crawled https://seekingalpha.com/news/3190157-amazons-streaming-video-starting-gain-traction
1495 . Crawled https://seekingalpha.com/news/1659093-report-google-looking-to-offer-mobile-services-in-fiber-markets
1496 . Crawled https://seekin

1563 . Crawled https://seekingalpha.com/news/1732073-google-buys-two-more-companies
1564 . Crawled https://seekingalpha.com/news/1697743-what-the-google-chief-exit-might-mean
1565 . Crawled https://seekingalpha.com/news/1694873-apple-google-intel-adobe-settle-no-hire-antitrust-suit
1566 . Crawled https://seekingalpha.com/news/1814053-googles-nest-labs-welcomes-third-party-apps
1567 . Crawled https://seekingalpha.com/news/1782883-apples-spotlight-search-dumps-google-in-favor-of-bing
1568 . Crawled https://seekingalpha.com/news/1765393-google-roundup-wi-fi-hardware-nest-recall-ad-plans
1569 . Crawled https://seekingalpha.com/news/2199505-gmail-access-partially-restored-in-china
1570 . Crawled https://seekingalpha.com/news/2257256-google-misses-by-0_20-misses-on-revenue
1571 . Crawled https://seekingalpha.com/news/1779843-wsj-google-to-spend-up-to-3b-on-satellite-network
1572 . Crawled https://seekingalpha.com/news/2350946-wsj-googles-phone-service-to-initially-have-just-one-giant-phone
1

1640 . Crawled https://seekingalpha.com/news/3217230-alphabets-self-driving-car-operation-become-stand-alone-business
1641 . Crawled https://seekingalpha.com/news/2059565-google-puts-pichai-in-charge-of-core-products
1642 . Crawled https://seekingalpha.com/news/3216601-google-fiber-ceo-departs-business-pauses-new-deployments
1643 . Crawled https://seekingalpha.com/news/2049355-google-tries-to-rethink-e-mail-with-inbox-app
1644 . Crawled https://seekingalpha.com/news/3216388-google-introduces-cloud-collaborative-whiteboard
1645 . Crawled https://seekingalpha.com/news/3216352-google-acquires-ar-vr-eye-tracking-startup
1646 . Crawled https://seekingalpha.com/news/3215991-googles-android-pay-targets-broadened-reach-partnerships-visa-mastercard
1647 . Crawled https://seekingalpha.com/news/3215808-autonomous-vehicle-talk-shifts-investing-matrix
1648 . Crawled https://seekingalpha.com/news/3215790-apple-monitoring-and-t-time-warner-media-consolidation-speeds
1649 . Crawled https://seekingalph

NameError: name 'no_err' is not defined

In [ ]:
results.to_csv('googlnews.csv')

The crawling task is quite a bit slow, you can wait up to 24 hours until it finishes crawling all things (~8k articles).



